In [1]:
import requests,json,os,time
apiKey=os.getenv('CKANAPIKEY')

In [2]:
mainDir = '/Users/iperezx/Documents/sage-commons/aot/'
sageCommonsURL='http://hotshot.sdsc.edu:5000'
headers = {'Authorization': apiKey}
orgName= 'array-of-things'

In [3]:
#get directories of datasets
directories = [f.path for f in os.scandir(mainDir) if f.is_dir() and f.name.startswith('chicago')]
print(directories)

['/Users/iperezx/Documents/sage-commons/aot/chicago-complete.monthly.2020-09-01-to-2020-09-30', '/Users/iperezx/Documents/sage-commons/aot/chicago-complete.daily.2020-10-25', '/Users/iperezx/Documents/sage-commons/aot/chicago-complete.weekly.2020-10-19-to-2020-10-25']


In [4]:
timeNames = ['daily','weekly','monthly']
dataDir = directories[1]
dataFiles = [f for f in sorted(os.listdir(dataDir)) if os.path.isfile(os.path.join(dataDir, f))] 
timeData = {key: None for key in timeNames}
for val in timeNames:
    timeData[val] = {key: None for key in dataFiles}
    

## import chicago daily data

### create data resource

In [5]:
apiAction='/api/3/action/package_create'
url = sageCommonsURL+apiAction

dataDir = directories[1]
title = dataDir.replace(mainDir,'')

timeType = 'daily'
name = 'chicago-' + timeType

tags = [{'name':'csv'},{'name':'waggle'},{'name':'sensors'},{'name': timeType}]

payload = {'owner_org': orgName,
           'title': title,
           'name' : name,
           'notes': 'Description of dataset',
           'tags' : tags
          }
response = requests.request("POST", url, headers=headers, json = payload)
jsonResponseDS = response.json()
print(response)
print(json.dumps(jsonResponseDS, indent=4, sort_keys=True))

<Response [200]>
{
    "help": "http://hotshot.sdsc.edu:5000/api/3/action/help_show?name=package_create",
    "result": {
        "author": null,
        "author_email": null,
        "creator_user_id": "abbb7828-b01e-41e4-b9fb-00413a8336ca",
        "extras": [],
        "groups": [],
        "id": "7800377a-0d70-4c59-a352-1a5963505a26",
        "isopen": false,
        "license_id": null,
        "license_title": null,
        "maintainer": null,
        "maintainer_email": null,
        "metadata_created": "2020-10-30T03:17:54.022552",
        "metadata_modified": "2020-10-30T03:17:54.022574",
        "name": "chicago-daily",
        "notes": "Description of dataset",
        "num_resources": 0,
        "num_tags": 4,
        "organization": {
            "approval_status": "approved",
            "created": "2020-10-30T01:55:38.413791",
            "description": "The Array of Things (AoT) is a collaborative effort among scientists, universities, \nfederallocal government,industry 

### create data source

In [6]:
apiAction='/api/3/action/resource_create'
packageID = jsonResponseDS['result']['id']
url = sageCommonsURL+apiAction

dataFiles = [f for f in sorted(os.listdir(dataDir)) if os.path.isfile(os.path.join(dataDir, f))]

for dataFile in dataFiles:
    dataPath= os.path.join(dataDir,dataFile)
    name = os.path.splitext(dataFile)[0]
    payload = {'package_id': packageID,
               'name' : name,
               'description': 'Description of data'
              }

    files = {'upload': open(dataPath, 'rb')}
    
    start = time.time() # start timing
    response = requests.request("POST", url, headers=headers, data = payload, files = files)
    end = time.time()
    timeData[timeType][dataFile] = abs(end-start)
    jsonResponse = response.json()
    print(response)
    print(json.dumps(jsonResponse, indent=4, sort_keys=True))
    
    print('Elapsed time: ' + str(timeData[timeType][dataFile]))

<Response [200]>
{
    "help": "http://hotshot.sdsc.edu:5000/api/3/action/help_show?name=resource_create",
    "result": {
        "cache_last_updated": null,
        "cache_url": null,
        "created": "2020-10-30T03:17:55.529379",
        "datastore_active": false,
        "description": "Description of data",
        "format": "",
        "hash": "",
        "id": "890e9c31-cc8a-4893-b65a-c648b12d7cd7",
        "last_modified": "2020-10-30T03:17:55.470064",
        "mimetype": null,
        "mimetype_inner": null,
        "name": "README",
        "package_id": "7800377a-0d70-4c59-a352-1a5963505a26",
        "position": 0,
        "resource_type": null,
        "revision_id": "22039359-1b0e-4c53-8ac0-4b70aab88708",
        "size": 6826,
        "state": "active",
        "url": "http://hotshot.sdsc.edu:5000/dataset/7800377a-0d70-4c59-a352-1a5963505a26/resource/890e9c31-cc8a-4893-b65a-c648b12d7cd7/download/readme.md",
        "url_type": "upload"
    },
    "success": true
}
Elapse

## import chicago weekly data

### create data resource

In [7]:
apiAction='/api/3/action/package_create'
url = sageCommonsURL+apiAction

dataDir = directories[2]
title = dataDir.replace(mainDir,'')

timeType = 'weekly'
name = 'chicago-' + timeType

tags = [{'name':'csv'},{'name':'waggle'},{'name':'sensors'},{'name': timeType}]

payload = {'owner_org': orgName,
           'title': title,
           'name' : name,
           'notes': 'Description of dataset',
           'tags' : tags
          }
response = requests.request("POST", url, headers=headers, json = payload)
jsonResponseDS = response.json()
print(response)
print(json.dumps(jsonResponseDS, indent=4, sort_keys=True))

<Response [200]>
{
    "help": "http://hotshot.sdsc.edu:5000/api/3/action/help_show?name=package_create",
    "result": {
        "author": null,
        "author_email": null,
        "creator_user_id": "abbb7828-b01e-41e4-b9fb-00413a8336ca",
        "extras": [],
        "groups": [],
        "id": "0faceaf8-b09e-4904-a5cb-a3e1c6824ca8",
        "isopen": false,
        "license_id": null,
        "license_title": null,
        "maintainer": null,
        "maintainer_email": null,
        "metadata_created": "2020-10-30T03:19:30.355754",
        "metadata_modified": "2020-10-30T03:19:30.355764",
        "name": "chicago-weekly",
        "notes": "Description of dataset",
        "num_resources": 0,
        "num_tags": 4,
        "organization": {
            "approval_status": "approved",
            "created": "2020-10-30T01:55:38.413791",
            "description": "The Array of Things (AoT) is a collaborative effort among scientists, universities, \nfederallocal government,industry

### create data source

In [8]:
apiAction='/api/3/action/resource_create'
packageID = jsonResponseDS['result']['id']
url = sageCommonsURL+apiAction

dataFiles = [f for f in sorted(os.listdir(dataDir)) if os.path.isfile(os.path.join(dataDir, f))]


for dataFile in dataFiles:
    dataPath= os.path.join(dataDir,dataFile)
    name = os.path.splitext(dataFile)[0]
    payload = {'package_id': packageID,
               'name' : name,
               'description': 'Description of data'
              }

    files = {'upload': open(dataPath, 'rb')}
    
    start = time.time() # start timing
    response = requests.request("POST", url, headers=headers, data = payload, files = files)
    end = time.time()
    timeData[timeType][dataFile] = abs(end-start)
    jsonResponse = response.json()
    print(response)
    print(json.dumps(jsonResponse, indent=4, sort_keys=True))
    
    print('Elapsed time: ' + str(timeData[timeType][dataFile]))

<Response [200]>
{
    "help": "http://hotshot.sdsc.edu:5000/api/3/action/help_show?name=resource_create",
    "result": {
        "cache_last_updated": null,
        "cache_url": null,
        "created": "2020-10-30T03:19:30.967414",
        "datastore_active": false,
        "description": "Description of data",
        "format": "",
        "hash": "",
        "id": "4f258853-1b88-4e91-9a4b-dfbb5f55084b",
        "last_modified": "2020-10-30T03:19:30.850506",
        "mimetype": null,
        "mimetype_inner": null,
        "name": "README",
        "package_id": "0faceaf8-b09e-4904-a5cb-a3e1c6824ca8",
        "position": 0,
        "resource_type": null,
        "revision_id": "8473c3db-1272-467c-a51b-0361d5971dad",
        "size": 6826,
        "state": "active",
        "url": "http://hotshot.sdsc.edu:5000/dataset/0faceaf8-b09e-4904-a5cb-a3e1c6824ca8/resource/4f258853-1b88-4e91-9a4b-dfbb5f55084b/download/readme.md",
        "url_type": "upload"
    },
    "success": true
}
Elapse

In [9]:
print(json.dumps(timeData, indent=4, sort_keys=True))

{
    "daily": {
        "README.md": 0.5854101181030273,
        "data.csv": 91.53000998497009,
        "nodes.csv": 0.7064409255981445,
        "offsets.csv": 0.6997570991516113,
        "provenance.csv": 0.6556820869445801,
        "sensors.csv": 0.7310867309570312
    },
    "monthly": {
        "README.md": null,
        "data.csv": null,
        "nodes.csv": null,
        "offsets.csv": null,
        "provenance.csv": null,
        "sensors.csv": null
    },
    "weekly": {
        "README.md": 0.5010290145874023,
        "data.csv": 722.333564043045,
        "nodes.csv": 1.0429399013519287,
        "offsets.csv": 0.7981760501861572,
        "provenance.csv": 0.8185009956359863,
        "sensors.csv": 0.8101160526275635
    }
}


## import chicago monthly data

### create dataset

In [10]:
apiAction='/api/3/action/package_create'
url = sageCommonsURL+apiAction

dataDir = directories[0]
title = dataDir.replace(mainDir,'')

timeType = 'monthly'
name = 'chicago-' + timeType

tags = [{'name':'csv'},{'name':'waggle'},{'name':'sensors'},{'name': timeType}]

payload = {'owner_org': orgName,
           'title': title,
           'name' : name,
           'notes': 'Description of dataset',
           'tags' : tags
          }
response = requests.request("POST", url, headers=headers, json = payload)
jsonResponseDS = response.json()
print(response)
print(json.dumps(jsonResponseDS, indent=4, sort_keys=True))

<Response [200]>
{
    "help": "http://hotshot.sdsc.edu:5000/api/3/action/help_show?name=package_create",
    "result": {
        "author": null,
        "author_email": null,
        "creator_user_id": "abbb7828-b01e-41e4-b9fb-00413a8336ca",
        "extras": [],
        "groups": [],
        "id": "fa095aab-7512-44f3-b388-8d6dec00e929",
        "isopen": false,
        "license_id": null,
        "license_title": null,
        "maintainer": null,
        "maintainer_email": null,
        "metadata_created": "2020-10-30T03:31:37.267665",
        "metadata_modified": "2020-10-30T03:31:37.267675",
        "name": "chicago-monthly",
        "notes": "Description of dataset",
        "num_resources": 0,
        "num_tags": 4,
        "organization": {
            "approval_status": "approved",
            "created": "2020-10-30T01:55:38.413791",
            "description": "The Array of Things (AoT) is a collaborative effort among scientists, universities, \nfederallocal government,industr

### create data source

In [12]:
apiAction='/api/3/action/resource_create'
packageID = jsonResponseDS['result']['id']
url = sageCommonsURL+apiAction

dataFiles = [f for f in sorted(os.listdir(dataDir)) if os.path.isfile(os.path.join(dataDir, f))]

for dataFile in dataFiles:
    dataPath= os.path.join(dataDir,dataFile)
    name = os.path.splitext(dataFile)[0]
    payload = {'package_id': packageID,
               'name' : name,
               'description': 'Description of data'
              }

    files = {'upload': open(dataPath, 'rb')}
    
    start = time.time() # start timing
    response = requests.request("POST", url, headers=headers, data = payload, files = files)
    end = time.time()
    timeData[timeType][dataFile] = abs(end-start)
#     jsonResponse = response.json()
#     print(response)
#     print(json.dumps(jsonResponse, indent=4, sort_keys=True))
    print('Elapsed time: ' + str(timeData[timeType][dataFile]))

Elapsed time: 3.071136951446533


KeyboardInterrupt: 